# **Predict RFC (Request for change) and possible failure / misconfiguration of ITSM assets**

## Importing necessary libraries

In [38]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,scale
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score,recall_score,classification_report,f1_score
import warnings
warnings.filterwarnings('ignore')

## Loading dataset 

In [2]:
data = pd.read_csv('C:\\Users\suchi\CDS Internship Datamites\ITSM_data.csv')
data

C:\Users\suchi\AppData\Local\Temp\ipykernel_3064\3081353933.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('C:\\Users\suchi\CDS Internship Datamites\ITSM_data.csv')


,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Status,Impact,Urgency,Priority,number_cnt,Category,KB_number,Alert_Status,No_of_Reassignments,Open_Time,Reopen_Time,Resolved_Time,Close_Time,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Incidents,No_of_Related_Changes,Related_Change
0,SUB000508,subapplication,Web Based Application,WBS000162,IM0000004,Closed,4,4,4.0,0.601292,incident,KM0000553,closed,26.0,05-02-2012 13:32,NaN,04-11-2013 13:50,04-11-2013 13:51,"3,87,16,91,111",Other,1.0,SD0000007,2.0,NaN,NaN
1,WBA000124,application,Web Based Application,WBS000088,IM0000005,Closed,3,3,3.0,0.415050,incident,KM0000611,closed,33.0,12-03-2012 15:44,02-12-2013 12:31,02-12-2013 12:36,02-12-2013 12:36,"4,35,47,86,389",Software,1.0,SD0000011,1.0,NaN,NaN
2,DTA000024,application,Desktop Application,WBS000092,IM0000006,Closed,NS,3,NaN,0.517551,request for information,KM0000339,closed,3.0,29-03-2012 12:36,NaN,13-01-2014 15:12,13-01-2014 15:13,"4,84,31,19,444",No error - works as designed,1.0,SD0000017,NaN,NaN,NaN
3,WBA000124,application,Web Based Application,WBS000088,IM0000011,Closed,4,4,4.0,0.642927,incident,KM0000611,closed,13.0,17-07-2012 11:49,NaN,14-11-2013 09:31,14-11-2013 09:31,"4,32,18,33,333",Operator error,1.0,SD0000025,NaN,NaN,NaN
4,WBA000124,application,Web Based Application,WBS000088,IM0000012,Closed,4,4,4.0,0.345258,incident,KM0000611,closed,2.0,10-08-2012 11:01,NaN,08-11-2013 13:55,08-11-2013 13:55,"3,38,39,03,333",Other,1.0,SD0000029,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46601,SBA000464,application,Server Based Application,WBS000073,IM0047053,Closed,4,4,4.0,0.231896,incident,KM0001314,closed,0.0,31-03-2014 16:23,NaN,31-03-2014 16:29,31-03-2014 16:29,"0,095",Other,1.0,SD0147021,NaN,NaN,NaN
46602,SBA000461,application,Server Based Application,WBS000073,IM0047054,Closed,4,4,4.0,0.805153,incident,KM0002360,closed,0.0,31-03-2014 15:03,NaN,31-03-2014 15:29,31-03-2014 15:29,"0,428333333",User error,1.0,SD0146967,NaN,NaN,NaN
46603,LAP000019,computer,Laptop,WBS000091,IM0047055,Closed,5,5,5.0,0.917466,incident,KM0000315,closed,0.0,31-03-2014 15:28,NaN,31-03-2014 15:32,31-03-2014 15:32,"0,071666667",Hardware,1.0,SD0146982,NaN,NaN,NaN
46604,WBA000058,application,Web Based Application,WBS000073,IM0047056,Closed,4,4,4.0,0.701278,incident,KM0001287,closed,0.0,31-03-2014 15:35,NaN,31-03-2014 15:42,31-03-2014 15:42,"0,116944444",Software,1.0,SD0146986,NaN,NaN,NaN


* As the above datetime columns are not in right format so pandas reads that column as object so we have to convert first those column into datetime.

In [4]:
#converting object column to datetime
data['Close_Time'] = pd.to_datetime(data['Close_Time'] ,format='mixed' )

In [5]:
data['Open_Time'] = pd.to_datetime(data['Open_Time'] ,format='mixed' )

In [6]:
data['Reopen_Time'] = pd.to_datetime(data['Reopen_Time'] ,format='mixed' )

In [7]:
data['Resolved_Time'] = pd.to_datetime(data['Resolved_Time'] ,format='mixed' )

## Basic checks 

In [8]:
data.head()

,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Status,Impact,Urgency,Priority,number_cnt,Category,KB_number,Alert_Status,No_of_Reassignments,Open_Time,Reopen_Time,Resolved_Time,Close_Time,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Incidents,No_of_Related_Changes,Related_Change
0,SUB000508,subapplication,Web Based Application,WBS000162,IM0000004,Closed,4,4,4.0,0.601292,incident,KM0000553,closed,26.0,2012-05-02 13:32:00,NaT,2013-04-11 13:50:00,2013-04-11 13:51:00,"3,87,16,91,111",Other,1.0,SD0000007,2.0,NaN,NaN
1,WBA000124,application,Web Based Application,WBS000088,IM0000005,Closed,3,3,3.0,0.415050,incident,KM0000611,closed,33.0,2012-12-03 15:44:00,2013-02-12 12:31:00,2013-02-12 12:36:00,2013-02-12 12:36:00,"4,35,47,86,389",Software,1.0,SD0000011,1.0,NaN,NaN
2,DTA000024,application,Desktop Application,WBS000092,IM0000006,Closed,NS,3,NaN,0.517551,request for information,KM0000339,closed,3.0,2012-03-29 12:36:00,NaT,2014-01-13 15:12:00,2014-01-13 15:13:00,"4,84,31,19,444",No error - works as designed,1.0,SD0000017,NaN,NaN,NaN
3,WBA000124,application,Web Based Application,WBS000088,IM0000011,Closed,4,4,4.0,0.642927,incident,KM0000611,closed,13.0,2012-07-17 11:49:00,NaT,2013-11-14 09:31:00,2013-11-14 09:31:00,"4,32,18,33,333",Operator error,1.0,SD0000025,NaN,NaN,NaN
4,WBA000124,application,Web Based Application,WBS000088,IM0000012,Closed,4,4,4.0,0.345258,incident,KM0000611,closed,2.0,2012-10-08 11:01:00,NaT,2013-08-11 13:55:00,2013-08-11 13:55:00,"3,38,39,03,333",Other,1.0,SD0000029,NaN,NaN,NaN


In [9]:
data.tail()

,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Status,Impact,Urgency,Priority,number_cnt,Category,KB_number,Alert_Status,No_of_Reassignments,Open_Time,Reopen_Time,Resolved_Time,Close_Time,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Incidents,No_of_Related_Changes,Related_Change
46601,SBA000464,application,Server Based Application,WBS000073,IM0047053,Closed,4,4,4.0,0.231896,incident,KM0001314,closed,0.0,2014-03-31 16:23:00,NaT,2014-03-31 16:29:00,2014-03-31 16:29:00,"0,095",Other,1.0,SD0147021,NaN,NaN,NaN
46602,SBA000461,application,Server Based Application,WBS000073,IM0047054,Closed,4,4,4.0,0.805153,incident,KM0002360,closed,0.0,2014-03-31 15:03:00,NaT,2014-03-31 15:29:00,2014-03-31 15:29:00,"0,428333333",User error,1.0,SD0146967,NaN,NaN,NaN
46603,LAP000019,computer,Laptop,WBS000091,IM0047055,Closed,5,5,5.0,0.917466,incident,KM0000315,closed,0.0,2014-03-31 15:28:00,NaT,2014-03-31 15:32:00,2014-03-31 15:32:00,"0,071666667",Hardware,1.0,SD0146982,NaN,NaN,NaN
46604,WBA000058,application,Web Based Application,WBS000073,IM0047056,Closed,4,4,4.0,0.701278,incident,KM0001287,closed,0.0,2014-03-31 15:35:00,NaT,2014-03-31 15:42:00,2014-03-31 15:42:00,"0,116944444",Software,1.0,SD0146986,NaN,NaN,NaN
46605,DCE000077,hardware,DataCenterEquipment,WBS000267,IM0047057,Closed,3,3,3.0,0.902320,incident,KM0000182,closed,0.0,2014-03-31 17:24:00,NaT,2014-03-31 22:47:00,2014-03-31 22:47:00,"0,586388889",Hardware,1.0,SD0147088,NaN,NaN,NaN


### examine the data

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46606 entries, 0 to 46605
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   CI_Name                     46606 non-null  object        
 1   CI_Cat                      46495 non-null  object        
 2   CI_Subcat                   46495 non-null  object        
 3   WBS                         46606 non-null  object        
 4   Incident_ID                 46606 non-null  object        
 5   Status                      46606 non-null  object        
 6   Impact                      46606 non-null  object        
 7   Urgency                     46606 non-null  object        
 8   Priority                    45226 non-null  float64       
 9   number_cnt                  46606 non-null  float64       
 10  Category                    46606 non-null  object        
 11  KB_number                   46606 non-null  object    

### Statistical measure of data

In [11]:
data.describe()

,Priority,number_cnt,No_of_Reassignments,Open_Time,Reopen_Time,Resolved_Time,Close_Time,No_of_Related_Interactions,No_of_Related_Incidents,No_of_Related_Changes
count,45226.000000,46606.000000,46605.000000,46606,2284,44826,46606,46492.000000,1222.000000,560.000000
mean,4.215805,0.499658,1.131831,2013-12-22 01:35:58.833626624,2013-12-19 15:54:41.847635968,2013-12-28 17:43:53.840182016,2013-12-28 08:05:15.215637248,1.149897,1.669394,1.058929
min,1.000000,0.000023,0.000000,2012-01-10 10:49:00,2013-01-10 09:58:00,2013-01-10 06:45:00,2013-01-10 06:45:00,1.000000,1.000000,1.000000
25%,4.000000,0.248213,0.000000,2013-10-22 11:24:15,2013-10-20 16:51:45,2013-10-25 17:15:00,2013-10-25 16:20:45,1.000000,1.000000,1.000000
50%,4.000000,0.500269,0.000000,2013-12-23 11:31:00,2013-12-17 10:31:00,2013-12-31 10:33:30,2013-12-31 08:43:00,1.000000,1.000000,1.000000
75%,5.000000,0.749094,2.000000,2014-03-01 10:46:15,2014-02-25 13:52:30,2014-03-13 10:39:45,2014-03-13 09:04:30,1.000000,1.000000,1.000000
max,5.000000,0.999997,46.000000,2014-12-03 22:58:00,2014-12-03 17:24:00,2014-12-03 17:56:00,2014-12-03 17:56:00,370.000000,63.000000,9.000000
std,0.705624,0.288634,2.269774,NaN,NaN,NaN,NaN,2.556338,3.339687,0.403596


* for this bussiness case our target column is No_of_Related_Changes

### Dropping unique column

In [13]:
data=data.drop(['Urgency','Impact','Alert_Status','No_of_Related_Incidents','Status','Open_Time','Reopen_Time','Close_Time','Resolved_Time'],axis=1)

In [14]:
data.head(2)

,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Priority,number_cnt,Category,KB_number,No_of_Reassignments,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Changes,Related_Change
0,SUB000508,subapplication,Web Based Application,WBS000162,IM0000004,4.0,0.601292,incident,KM0000553,26.0,"3,87,16,91,111",Other,1.0,SD0000007,NaN,NaN
1,WBA000124,application,Web Based Application,WBS000088,IM0000005,3.0,0.415050,incident,KM0000611,33.0,"4,35,47,86,389",Software,1.0,SD0000011,NaN,NaN


### Checking for missing value

In [15]:
data.isnull().sum()

CI_Name                           0
CI_Cat                          111
CI_Subcat                       111
WBS                               0
Incident_ID                       0
Priority                       1380
number_cnt                        0
Category                          0
KB_number                         0
No_of_Reassignments               1
Handle_Time_hrs                   1
Closure_Code                    460
No_of_Related_Interactions      114
Related_Interaction               0
No_of_Related_Changes         46046
Related_Change                46046
dtype: int64

In [18]:
data['No_of_Related_Changes'].fillna(method = 'ffill' , inplace = True )

In [22]:
data.isnull().sum().to_frame().T

,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Priority,number_cnt,Category,KB_number,No_of_Reassignments,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Changes,Related_Change
0,0,111,111,0,0,1380,0,0,0,1,1,460,114,0,10,46046


In [23]:
data['No_of_Related_Changes'].fillna(method = 'bfill' , inplace = True )

In [24]:
data.isnull().sum().to_frame().T

,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Priority,number_cnt,Category,KB_number,No_of_Reassignments,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Changes,Related_Change
0,0,111,111,0,0,1380,0,0,0,1,1,460,114,0,0,46046


In [25]:
data['Related_Change'].fillna(method = 'ffill' , inplace = True )

In [26]:
data.isnull().sum().to_frame().T

,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Priority,number_cnt,Category,KB_number,No_of_Reassignments,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Changes,Related_Change
0,0,111,111,0,0,1380,0,0,0,1,1,460,114,0,0,10


In [27]:
data['Related_Change'].fillna(method = 'bfill' , inplace = True )

In [28]:
data.isnull().sum().to_frame().T

,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Priority,number_cnt,Category,KB_number,No_of_Reassignments,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Changes,Related_Change
0,0,111,111,0,0,1380,0,0,0,1,1,460,114,0,0,0


In [29]:
data.dropna(inplace = True , how = 'any')

In [30]:
data.isnull().sum().to_frame().T

,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Priority,number_cnt,Category,KB_number,No_of_Reassignments,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Changes,Related_Change
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Define X and y

In [31]:
X=data.loc[:,data.columns!='No_of_Related_Changes']
y=data.No_of_Related_Changes


## Encoding the data

In [32]:
enc=LabelEncoder()
X.CI_Name=enc.fit_transform(X.CI_Name)
X.CI_Cat=enc.fit_transform(X.CI_Cat)
X.Incident_ID=enc.fit_transform(X.Incident_ID)
X.CI_Subcat=enc.fit_transform(X.CI_Subcat)
X.WBS=enc.fit_transform(X.WBS)
X.Category=enc.fit_transform(X.Category)
X.KB_number=enc.fit_transform(X.KB_number)
X.Related_Interaction=enc.fit_transform(X.Related_Interaction)
X.Handle_Time_hrs=enc.fit_transform(X.Handle_Time_hrs)
X.Closure_Code=enc.fit_transform(X.Closure_Code)
X.Related_Change=enc.fit_transform(X.Related_Change)
X.head()

,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Priority,number_cnt,Category,KB_number,No_of_Reassignments,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,Related_Change
0,2426,11,55,135,0,4.0,0.601292,1,404,26.0,19414,6,1.0,1,2
1,2542,1,55,70,1,3.0,0.415050,1,447,33.0,20902,10,1.0,2,2
3,2542,1,55,70,2,4.0,0.642927,1,447,13.0,20818,5,1.0,3,2
4,2542,1,55,70,3,4.0,0.345258,1,447,2.0,17912,6,1.0,4,2
5,2542,1,55,70,4,4.0,0.006676,1,447,4.0,17909,6,1.0,5,2


## Splitting the data

In [33]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=20,test_size=0.3)

In [34]:
print("X_train shape = ",X_train.shape)
print("X_test shape = ",X_test.shape)
print("y_train shape = ",y_train.shape)
print("y_test shape = ",y_test.shape)

X_train shape =  (31187, 15)
X_test shape =  (13367, 15)
y_train shape =  (31187,)
y_test shape =  (13367,)


## 1. Random Forest Classifier

In [35]:
model=RandomForestClassifier(n_estimators=250,random_state=10,max_depth=4)
model.fit(X_train,y_train)

RandomForestClassifier(max_depth=4, n_estimators=250, random_state=10)

In [36]:
y_train_predict=model.predict(X_train)
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict)*100)

99.76060447370389


In [39]:
print("\n Accuracy of Training = " ,accuracy_score(y_train,y_train_predict)*100)
print("\n Accuracy of Testing =  ", accuracy_score(y_test,y_predict)*100)
print("\n Precision score: {}".format(precision_score(y_test,y_predict,average='weighted')*100))
print("\n Recall score: {}".format(recall_score(y_test,y_predict,average='weighted')*100))
print("\n F1 score: {}".format(f1_score(y_test,y_predict,average='weighted')*100))



 Accuracy of Training =  99.78837336069516

 Accuracy of Testing =   99.76060447370389

 Precision score: 99.52561828105654

 Recall score: 99.76060447370389

 F1 score: 99.64202490572153


In [40]:
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00     12663
         2.0       0.97      1.00      0.99       672
         3.0       0.00      0.00      0.00        29
         9.0       0.00      0.00      0.00         3

    accuracy                           1.00     13367
   macro avg       0.49      0.50      0.50     13367
weighted avg       1.00      1.00      1.00     13367



In [41]:
pd.crosstab(y_predict,y_test)

No_of_Related_Changes,1.0,2.0,3.0,9.0
row_0,,,,
1.0,12663,0,12,0
2.0,0,672,17,3


## 2. XGB Classifier

In [43]:
class_mapping = {1: 0, 2: 1, 3: 2 , 9:3}
y_train_mapped = y_train.map(class_mapping)
y_test_mapped = y_test.map(class_mapping)

In [44]:
model=XGBClassifier(max_depth=3,learning_rate=0.5,random_state=50,n_estimators=50)
model.fit(X_train,y_train_mapped)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.5, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=50, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [45]:
y_train_predict=model.predict(X_train)
y_predict=model.predict(X_test)

In [47]:
print("\nTrain accuracy = " ,accuracy_score(y_train_mapped,y_train_predict)*100)
print("\nTest accuracy = " ,accuracy_score(y_test_mapped,y_predict)*100)
print("\nPrecision score: {}".format(precision_score(y_test_mapped,y_predict,average='weighted')*100))
print("\nRecall score: {}".format(recall_score(y_test_mapped,y_predict,average='weighted')*100))
print("\nF1 score: {}".format(f1_score(y_test_mapped,y_predict,average='weighted')*100))


Train accuracy =  100.0

Test accuracy =  99.96259444901622

Precision score: 99.96209343219027

Recall score: 99.96259444901622

F1 score: 99.96228023555724


In [48]:
pd.crosstab(y_test_mapped,y_predict)

col_0,0,1,2,3
No_of_Related_Changes,,,,
0,12663,0,0,0
1,0,670,2,0
2,0,3,26,0
3,0,0,0,3


In [50]:
print(classification_report(y_test_mapped,y_predict))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12663
           1       1.00      1.00      1.00       672
           2       0.93      0.90      0.91        29
           3       1.00      1.00      1.00         3

    accuracy                           1.00     13367
   macro avg       0.98      0.97      0.98     13367
weighted avg       1.00      1.00      1.00     13367



* Both the model perform very well with accuaracy with 99%